In [2]:
import requests
import pandas as pd
pd.options.display.max_columns=200
import matplotlib.pyplot as plt

In [3]:
url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=max&interval=daily"
response = requests.get(url)
data = response.json()

In [4]:
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
df.drop('timestamp', axis=1, inplace=True)

In [5]:
df.drop(df.index[-1], axis = 0, inplace=True)

In [6]:
df['progression daily']=0

In [7]:
df

,price,date,progression daily
0,135.300000,2013-04-28,0
1,141.960000,2013-04-29,0
2,135.300000,2013-04-30,0
3,117.000000,2013-05-01,0
4,103.430000,2013-05-02,0
...,...,...,...
3811,27435.874615,2023-10-06,0
3812,27958.196437,2023-10-07,0
3813,27977.543491,2023-10-08,0
3814,27948.103652,2023-10-09,0


In [8]:
#Progression daily : improvement on the previous day
for i in range(df.shape[0] - 1):
      df.loc[i+1, 'progression daily'] = (df['price'][i+1] / df['price'][i]) - 1

C:\Users\thoma\AppData\Local\Temp\ipykernel_6208\3522090179.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.04922394678492226' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i+1, 'progression daily'] = (df['price'][i+1] / df['price'][i]) - 1


In [9]:
df

,price,date,progression daily
0,135.300000,2013-04-28,0.000000
1,141.960000,2013-04-29,0.049224
2,135.300000,2013-04-30,-0.046915
3,117.000000,2013-05-01,-0.135255
4,103.430000,2013-05-02,-0.115983
...,...,...,...
3811,27435.874615,2023-10-06,-0.012818
3812,27958.196437,2023-10-07,0.019038
3813,27977.543491,2023-10-08,0.000692
3814,27948.103652,2023-10-09,-0.001052


In [10]:
#Progression daily : improvement on the next day (predict target)

df['progression tomorrow']=0.0
for i in range(df.shape[0] - 1):
      df.loc[i, 'progression tomorrow'] = (df['price'][i+1] / df['price'][i]) - 1

In [11]:
df

,price,date,progression daily,progression tomorrow
0,135.300000,2013-04-28,0.000000,0.049224
1,141.960000,2013-04-29,0.049224,-0.046915
2,135.300000,2013-04-30,-0.046915,-0.135255
3,117.000000,2013-05-01,-0.135255,-0.115983
4,103.430000,2013-05-02,-0.115983,-0.120081
...,...,...,...,...
3811,27435.874615,2023-10-06,-0.012818,0.019038
3812,27958.196437,2023-10-07,0.019038,0.000692
3813,27977.543491,2023-10-08,0.000692,-0.001052
3814,27948.103652,2023-10-09,-0.001052,-0.012678


In [12]:
#Fear and greed index

# URL de base
base_url = "https://api.alternative.me"

# Endpoint pour le Fear and Greed Index
endpoint = "/fng/"

# Paramètres
params = {
    "limit": "0",  # obtenir toutes les données disponibles
    "format": "json",  # format des données
    "date_format": "world"  # format de la date
}

# Faire la requête
response = requests.get(base_url + endpoint, params=params)

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()["data"]
    df_fng = pd.DataFrame(data)
else:
    print(f"Erreur {response.status_code}: {response.text}")

In [13]:
df_fng.drop(df_fng.columns[-1], axis=1, inplace=True)

In [14]:
df_fng

,value,value_classification,timestamp
0,50,Neutral,10-10-2023
1,50,Neutral,09-10-2023
2,50,Neutral,08-10-2023
3,49,Neutral,07-10-2023
4,50,Neutral,06-10-2023
...,...,...,...
2070,11,Extreme Fear,05-02-2018
2071,24,Extreme Fear,04-02-2018
2072,40,Fear,03-02-2018
2073,15,Extreme Fear,02-02-2018


In [15]:
# Convertir la colonne 'date' en datetime pour les deux DataFrames
df['date'] = pd.to_datetime(df['date'])
df_fng['date'] = pd.to_datetime(df_fng['timestamp'], dayfirst=True)

# Fusionner les deux DataFrames sur la colonne 'date'
df_merged = df.merge(df_fng, on='date', how='left')

In [16]:
df_merged

,price,date,progression daily,progression tomorrow,value,value_classification,timestamp
0,135.300000,2013-04-28,0.000000,0.049224,NaN,NaN,NaN
1,141.960000,2013-04-29,0.049224,-0.046915,NaN,NaN,NaN
2,135.300000,2013-04-30,-0.046915,-0.135255,NaN,NaN,NaN
3,117.000000,2013-05-01,-0.135255,-0.115983,NaN,NaN,NaN
4,103.430000,2013-05-02,-0.115983,-0.120081,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3811,27435.874615,2023-10-06,-0.012818,0.019038,50,Neutral,06-10-2023
3812,27958.196437,2023-10-07,0.019038,0.000692,49,Neutral,07-10-2023
3813,27977.543491,2023-10-08,0.000692,-0.001052,50,Neutral,08-10-2023
3814,27948.103652,2023-10-09,-0.001052,-0.012678,50,Neutral,09-10-2023


In [17]:
import requests

# URL de l'API GraphQL Bitquery
url = "https://graphql.bitquery.io/"

# Votre clé d'API
api_key = "BQYfg1D6vThVHnkkxUH0jEXNw3ZrwQ70"

# Requête GraphQL pour obtenir l'historique du hash rate
query = """
{
  bitcoin {
    statistics {
      hashrate {
        timestamp
        value
      }
    }
  }
}
"""

# Créer un dictionnaire d'en-tête avec la clé d'API
headers = {"X-API-KEY": api_key}

# Envoi de la requête avec l'en-tête d'authentification
response = requests.post(url, json={"query": query}, headers=headers)

# Vérification de la réponse
if response.status_code == 200:
    data = response.json()
    hashrate = pd.DataFrame(data)
else:
    print(f"Erreur {response.status_code}: {response.text}")


In [21]:
hashrate

,errors
0,{'message': 'Field 'statistics' doesn't exist ...


In [19]:
import numpy
import scipy

In [20]:
from sklearn.linear_model import LogisticRegression